In [35]:
import numpy as np
import pandas as pd
import datetime as dt
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import ImageGrid
from os import listdir, makedirs
from os.path import join, exists, expanduser
from sklearn.metrics import log_loss, accuracy_score
from sklearn.metrics import log_loss, accuracy_score
from keras.preprocessing import image
from keras.applications.vgg16 import VGG16
from keras.applications.resnet50 import ResNet50
from keras.applications import xception
from keras.applications import inception_v3
from keras.applications.vgg16 import preprocess_input, decode_predictions
from sklearn.linear_model import LogisticRegression

In [36]:
INPUT_SIZE = 224
NUM_CLASSES = 16
SEED = 1987

In [37]:
data_dir = '/u/13/lix12/unix/Downloads/dogbreed'
!ls /u/13/lix12/unix/Downloads/dogbreed

labels.csv  sample_submission.csv  test  train


In [61]:
labels = pd.read_csv(join(data_dir, 'labels.csv'))
labels.head(5)

,id,breed
0,000bec180eb18c7604dcecc8fe0dba07,boston_bull
1,001513dfcb2ffafc82cccf4d8bbaba97,dingo
2,001cdf01b096e06d78e9e5112d419397,pekinese
3,00214f311d5d2247d5dfe4fe24b2303d,bluetick
4,0021f9ceb3235effd7fcde7f7538ed62,golden_retriever


In [62]:
list(labels.groupby('breed'))

[('affenpinscher',                                      id          breed
  30     00ca18751837cd6a22813f8e221f7819  affenpinscher
  44     01268f0007876a692907bda89468184c  affenpinscher
  56     0162107acd8f2588c0944b791d61bb0c  affenpinscher
  172    040fef64640b89c53f161b6c5215b78b  affenpinscher
  194    0484a6cc686a07e1edacd0fb44b1d965  affenpinscher
  658    100f347ec78a42a9e7c2418e4beb3f6a  affenpinscher
  673    106d7e0df869e7ba9f6d16c5f77ddfb8  affenpinscher
  691    10edc6f98c2423e4e195e09a86b0d292  affenpinscher
  719    11b60d8d86f14a601ca290909a17cbc6  affenpinscher
  1002   18b4e732c105272f40956c2f85813bcb  affenpinscher
  1036   19a8c1698819d382c3ead14bdc1a360b  affenpinscher
  1041   19de1db12b3ddc7f2af6d9453c977083  affenpinscher
  1045   1a012faf98b32039adc17ee708bcb360  affenpinscher
  1076   1aa98488ce65f9b57b468ccde4f1cad8  affenpinscher
  1249   1ee0c041d15647e3909625eba419f153  affenpinscher
  1300   20430175869defec1722d3432f42ebd7  affenpinscher
  1375   21cc2

In [63]:
sample_submission = pd.read_csv(join(data_dir, 'sample_submission.csv'))
sample_submission.head(5)

,id,affenpinscher,afghan_hound,african_hunting_dog,airedale,american_staffordshire_terrier,appenzeller,australian_terrier,basenji,basset,...,toy_poodle,toy_terrier,vizsla,walker_hound,weimaraner,welsh_springer_spaniel,west_highland_white_terrier,whippet,wire-haired_fox_terrier,yorkshire_terrier
0,000621fb3cbb32d8935728e48679680e,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,...,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333
1,00102ee9d8eb90812350685311fe5890,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,...,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333
2,0012a730dfa437f5f3613fb75efcd4ce,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,...,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333
3,001510bc8570bbeee98c8d80c8a95ec1,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,...,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333
4,001a5f3114548acdefa3d4da05474c2e,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,...,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333


In [40]:
print(len(listdir(join(data_dir, 'train'))), len(labels))
print(len(listdir(join(data_dir, 'test'))), len(sample_submission))

10222 10222
10357 10357


Sort by group of greed the number of 'scottish_deerhound' is biggest.

In [64]:
selected_breed_list = list(labels.groupby('breed').count().sort_values(by='id', ascending=False).head(NUM_CLASSES).index)
selected_breed_list

['scottish_deerhound',
 'maltese_dog',
 'afghan_hound',
 'entlebucher',
 'bernese_mountain_dog',
 'shih-tzu',
 'great_pyrenees',
 'pomeranian',
 'basenji',
 'samoyed',
 'airedale',
 'tibetan_terrier',
 'leonberg',
 'cairn',
 'beagle',
 'japanese_spaniel']

In [65]:
labels['breed'].isin(selected_breed_list)

0        False
1        False
2        False
3        False
4        False
5        False
6        False
7        False
8         True
9         True
10       False
11       False
12        True
13       False
14       False
15       False
16       False
17       False
18       False
19       False
20       False
21       False
22       False
23       False
24       False
25       False
26       False
27       False
28       False
29        True
         ...  
10192    False
10193    False
10194    False
10195     True
10196    False
10197    False
10198    False
10199    False
10200     True
10201    False
10202    False
10203    False
10204     True
10205    False
10206    False
10207     True
10208    False
10209    False
10210    False
10211    False
10212     True
10213    False
10214    False
10215     True
10216     True
10217    False
10218    False
10219     True
10220    False
10221    False
Name: breed, Length: 10222, dtype: bool

In [73]:
labels = labels[labels['breed'].isin(selected_breed_list)]
labels.head(5)

,id,breed,target,rank
8,003df8b8a8b05244b1d920bb6cf451f9,basenji,1,1.0
9,0042188c895a2f14ef64a918ed9c7b64,scottish_deerhound,1,1.0
12,00693b8bc2470375cc744a6391d397ec,maltese_dog,1,1.0
29,00bee065dcec471f26394855c5c2f3de,cairn,1,1.0
48,013f8fdf6d638c7bb042f5f17e8a9fdc,tibetan_terrier,1,1.0


In [59]:
labels

,id,breed,target,rank
8,003df8b8a8b05244b1d920bb6cf451f9,basenji,1,1.0
9,0042188c895a2f14ef64a918ed9c7b64,scottish_deerhound,1,1.0
12,00693b8bc2470375cc744a6391d397ec,maltese_dog,1,1.0
29,00bee065dcec471f26394855c5c2f3de,cairn,1,1.0
48,013f8fdf6d638c7bb042f5f17e8a9fdc,tibetan_terrier,1,1.0
57,0162b3e0144fb1d1ab82fbff3ace9938,shih-tzu,1,1.0
70,01b36cb1b80ab8c3a7d2b7128ad21bdc,samoyed,1,1.0
79,01e787576c003930f96c966f9c3e1d44,scottish_deerhound,1,2.0
80,01ee3c7ff9bcaba9874183135877670e,entlebucher,1,1.0
88,021b5a49189665c0442c19b5b33e8cf1,entlebucher,1,2.0


In [71]:
labels['target'] = 1
labels

,id,breed,target,rank
8,003df8b8a8b05244b1d920bb6cf451f9,basenji,1,1.0
9,0042188c895a2f14ef64a918ed9c7b64,scottish_deerhound,1,1.0
12,00693b8bc2470375cc744a6391d397ec,maltese_dog,1,1.0
29,00bee065dcec471f26394855c5c2f3de,cairn,1,1.0
48,013f8fdf6d638c7bb042f5f17e8a9fdc,tibetan_terrier,1,1.0
57,0162b3e0144fb1d1ab82fbff3ace9938,shih-tzu,1,1.0
70,01b36cb1b80ab8c3a7d2b7128ad21bdc,samoyed,1,1.0
79,01e787576c003930f96c966f9c3e1d44,scottish_deerhound,1,2.0
80,01ee3c7ff9bcaba9874183135877670e,entlebucher,1,1.0
88,021b5a49189665c0442c19b5b33e8cf1,entlebucher,1,2.0


In [74]:
labels['rank'] = labels.groupby('breed').rank()['id']


TypeError: 'SeriesGroupBy' object is not callable